In [27]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import root_mean_squared_error
from sklearn.model_selection import KFold
from sklearn.cross_decomposition import CCA
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import statistics

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
cca = pd.read_csv('/content/drive/MyDrive/UW/sphscAudiogram/audiogram_cca.csv')

In [29]:
predictors = ['AGE', 'GENDER_Female', 'GENDER_Male', 'LT1', 'RT1', 'L250', 'L500']
X = cca[predictors]
Y = cca.drop(columns = predictors)

In [30]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=6)

In [33]:
cca_model = CCA(n_components=5)
cca_model.fit(X_train, Y_train)

X_cca, Y_cca = cca_model.transform(X_train, Y_train)
score = cca_model.score(X_test, Y_test)
print(score)

0.3989256180016808


In [34]:
root_mean_squared_error(cca_model.predict(X_test), Y_test)

16.64595245129827

## CV

In [24]:
def cca_cv(predictors, data, n_splits = 10, shuffle = True, random_state = 42):
  cv = KFold(n_splits=n_splits, shuffle=shuffle, random_state=random_state)
  X = data[predictors]
  Y = data.drop(columns = predictors)
  rmse = {}
  for n_comp in range(1, len(predictors)):
    rmse[n_comp] = []
    for train_index, test_index in cv.split(X):
      X_train, X_test = X.iloc[train_index], X.iloc[test_index]
      Y_train, Y_test = Y.iloc[train_index], Y.iloc[test_index]

      cca_model = CCA(n_components=n_comp)
      cca_model.fit(X_train, Y_train)
      rmse[n_comp].append(root_mean_squared_error(cca_model.predict(X_test), Y_test))

  return rmse

In [25]:
rmse = cca_cv(predictors, cca)

In [28]:
{key: statistics.mean(value) for key, value in rmse.items()}

{1: 14.094567291119887,
 2: 14.398533052646007,
 3: 15.110606007150562,
 4: 16.792411817525764}